In [2]:
import os
import pandas as pd
import numpy as np

## Response to reviewer 3

Comment: The link prediction is a ranking problem. In Section 3.2, the authors mention that they frame it as a multi-class classification problem. This assumes that only one tail entity (class) is valid and the rest is invalid, which is ok for n-to-1 relations but wrong for 1-to-n and n-to-n relations. For example, the relationship occupation for a person could have multiple true values for a person with a long career or multiple affiliations. The temporality of the triples/relations is something that could be considered as well in the future.

My idea: Check in how many instances a person in the test set has more than one occupation.

1. Load the test dataset of Humanw5M-3Mil

In [2]:
dataset_HW5M3M = pd.read_csv('/home/lena/git/master_thesis_bias_in_NLP/results/bias_measurement/link_prediction_bias/preds_df_occupation_HumanW5M_subset_testset_with_sensitive_attribute_info.tsv',
                         sep = '\t')
dataset_HW5M3M.head()

,head_id,relation_id,relation_label,true_tail_id,true_tail_label,true_tail_class_label,P21,P27,P172,P140
0,Q207578,P106,occupation,Q28389,screenwriter,8,-1,Q142,-1,-1
1,Q862464,P106,occupation,Q753110,songwriter,0,Q6581097,-1,-1,-1
2,Q5996937,P106,occupation,Q82955,politician,1,Q6581072,-1,-1,-1
3,Q6117411,P106,occupation,Q37226,teacher,0,-1,Q30,-1,-1
4,Q12983,P106,occupation,Q82955,politician,1,-1,-1,-1,-1


In [17]:
dataset_HW5M3M.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3763 entries, 0 to 3762
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   head_id                3763 non-null   object
 1   relation_id            3763 non-null   object
 2   relation_label         3763 non-null   object
 3   true_tail_id           3763 non-null   object
 4   true_tail_label        3763 non-null   object
 5   true_tail_class_label  3763 non-null   int64 
 6   P21                    3763 non-null   object
 7   P27                    3763 non-null   object
 8   P172                   3763 non-null   object
 9   P140                   3763 non-null   object
dtypes: int64(1), object(9)
memory usage: 294.1+ KB


2. Count the occurrences of `head_id` and check whether the count is ever > 1.

In [8]:
value_counts_head_id = dataset_HW5M3M['head_id'].value_counts()

In [19]:
IDs_and_counts_above_one_ocupation = value_counts_head_id[value_counts_head_id > 1]
IDs_and_counts_above_one_ocupation

Q44414       3
Q6780980     2
Q47755       2
Q3242404     2
Q107424      2
Q4228038     2
Q804182      2
Q215610      2
Q354241      2
Q2713565     2
Q611182      2
Q7825274     2
Q774923      2
Q5541005     2
Q3172990     2
Q28791080    2
Q492743      2
Q58793       2
Q236370      2
Q316600      2
Q4720002     2
Q379923      2
Q366328      2
Q1933451     2
Q405463      2
Q318462      2
Q7817666     2
Q17512078    2
Q6850043     2
Q971395      2
Q332470      2
Q2150133     2
Q835766      2
Q231942      2
Q1274954     2
Q695938      2
Q5267978     2
Q1067        2
Name: head_id, dtype: int64

In [16]:
IDs_and_counts_above_one_ocupation.info()

<class 'pandas.core.series.Series'>
Index: 38 entries, Q44414 to Q1067
Series name: head_id
Non-Null Count  Dtype
--------------  -----
38 non-null     int64
dtypes: int64(1)
memory usage: 1.6+ KB


In [26]:
people_with_more_than_one_occupation = IDs_and_counts_above_one_ocupation.index.values
people_with_more_than_one_occupation

array(['Q44414', 'Q6780980', 'Q47755', 'Q3242404', 'Q107424', 'Q4228038',
       'Q804182', 'Q215610', 'Q354241', 'Q2713565', 'Q611182', 'Q7825274',
       'Q774923', 'Q5541005', 'Q3172990', 'Q28791080', 'Q492743',
       'Q58793', 'Q236370', 'Q316600', 'Q4720002', 'Q379923', 'Q366328',
       'Q1933451', 'Q405463', 'Q318462', 'Q7817666', 'Q17512078',
       'Q6850043', 'Q971395', 'Q332470', 'Q2150133', 'Q835766', 'Q231942',
       'Q1274954', 'Q695938', 'Q5267978', 'Q1067'], dtype=object)

3. For the most accurate of our models, SimKGC_IBSNPB: Are there instances where false positives should actually be true positives?

In [35]:
preds_df_SKGC_IBSNPB = pd.read_csv('/home/lena/git/master_thesis_bias_in_NLP/results/bias_measurement/link_prediction_bias/SimKGC_IBSNPB/preds_df_SimKGC_IBSNPB_HW5M_occupation_9classes_WITHOUTNAs_sensrel_from_entire_v4subset.tsv',
                         sep = '\t')
preds_df_SKGC_IBSNPB.head()

,head_id,relation_id,relation_label,true_tail_id,true_tail_label,true_tail_class_label,pred_tail_id,pred_tail_label,pred_tail_class_label,P21,P27,P172,P140
0,Q207578,P106,occupation,Q28389,screenwriter,8,Q4610556,model,0,-1,Q142,-1,-1
1,Q862464,P106,occupation,Q753110,songwriter,0,Q753110,songwriter,0,Q6581097,-1,-1,-1
2,Q5996937,P106,occupation,Q82955,politician,1,Q82955,politician,1,Q6581072,-1,-1,-1
3,Q6117411,P106,occupation,Q37226,teacher,0,Q36180,writer,2,-1,Q30,-1,-1
4,Q12983,P106,occupation,Q82955,politician,1,Q2478141,aristocrat,0,-1,-1,-1,-1


In [36]:
preds_df_onlymultiocc = preds_df_SKGC_IBSNPB[preds_df_SKGC_IBSNPB['head_id'].isin(people_with_more_than_one_occupation)]

In [37]:
preds_df_onlymultiocc = preds_df_onlymultiocc.sort_values('head_id')
preds_df_onlymultiocc

,head_id,relation_id,relation_label,true_tail_id,true_tail_label,true_tail_class_label,pred_tail_id,pred_tail_label,pred_tail_class_label,P21,P27,P172,P140
2916,Q1067,P106,occupation,Q4964182,philosopher,0,Q670152,Germanophile,0,-1,Q148540,-1,-1
1518,Q1067,P106,occupation,Q58968,intellectual,0,Q670152,Germanophile,0,-1,Q148540,-1,-1
2070,Q107424,P106,occupation,Q486748,pianist,0,Q488205,singer-songwriter,0,-1,-1,-1,-1
912,Q107424,P106,occupation,Q639669,musician,0,Q488205,singer-songwriter,0,-1,-1,-1,-1
722,Q1274954,P106,occupation,Q1930187,journalist,0,Q2722764,radio personality,0,Q6581097,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,Q804182,P106,occupation,Q82955,politician,1,Q876864,contributing editor,0,-1,-1,-1,-1
1065,Q835766,P106,occupation,Q6625963,novelist,7,Q6625963,novelist,7,-1,-1,-1,-1
2383,Q835766,P106,occupation,Q622807,seiyū,0,Q6625963,novelist,7,-1,-1,-1,-1
485,Q971395,P106,occupation,Q82955,politician,1,Q82955,politician,1,-1,-1,-1,-1


For SimKGC_IBSNPB, 73 rows are potentially affected. But how many of these rows are actual misclassifiations, i.e. potential false positives?

In [44]:
preds_df_onlymultiocc[preds_df_onlymultiocc['true_tail_class_label'] == preds_df_onlymultiocc['pred_tail_class_label']]

,head_id,relation_id,relation_label,true_tail_id,true_tail_label,true_tail_class_label,pred_tail_id,pred_tail_label,pred_tail_class_label,P21,P27,P172,P140
2916,Q1067,P106,occupation,Q4964182,philosopher,0,Q670152,Germanophile,0,-1,Q148540,-1,-1
1518,Q1067,P106,occupation,Q58968,intellectual,0,Q670152,Germanophile,0,-1,Q148540,-1,-1
2070,Q107424,P106,occupation,Q486748,pianist,0,Q488205,singer-songwriter,0,-1,-1,-1,-1
912,Q107424,P106,occupation,Q639669,musician,0,Q488205,singer-songwriter,0,-1,-1,-1,-1
722,Q1274954,P106,occupation,Q1930187,journalist,0,Q2722764,radio personality,0,Q6581097,-1,-1,-1
2889,Q17512078,P106,occupation,Q131512,farmer,0,Q131512,farmer,0,-1,-1,-1,-1
2444,Q2150133,P106,occupation,Q16533,judge,0,Q883231,law clerk,0,-1,-1,-1,-1
2812,Q231942,P106,occupation,Q486748,pianist,0,Q488205,singer-songwriter,0,-1,-1,-1,-1
2652,Q231942,P106,occupation,Q753110,songwriter,0,Q488205,singer-songwriter,0,-1,-1,-1,-1
315,Q236370,P106,occupation,Q753110,songwriter,0,Q855091,guitarist,0,-1,Q262,-1,-1


First: There are in total 37 misclassifications.

During the sampling of Wikidata5M a random approach is taken, leaving a few questions. In the HUMANW5M-3MIL dataset, it's mentioned that there is no female screenwriter, **but were there any in the initial 11M triples?** If yes, this implies that some level of bias was already introduced during the creation of the proposed dataset.

In [4]:
# load each split
HW5M3MIL_train = pd.read_csv('/home/lena/git/master_thesis_bias_in_NLP/data/processed/output_of_preprocessing/training_data_subset_0.9_rs42_06_05_2022_15:11.tsv',
                         sep = '\t', names = ['head_entity', 'relation', 'tail_entity'])
HW5M3MIL_train.head()
HW5M3MIL_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2791044 entries, 0 to 2791043
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   head_entity  object
 1   relation     object
 2   tail_entity  object
dtypes: object(3)
memory usage: 63.9+ MB


In [5]:
# load each split
HW5M3MIL_val = pd.read_csv('/home/lena/git/master_thesis_bias_in_NLP/data/processed/output_of_preprocessing/validation_data_subset_0.05_rs42_06_05_2022_15:11.tsv',
                         sep = '\t', names = ['head_entity', 'relation', 'tail_entity'])
HW5M3MIL_val.head()
HW5M3MIL_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155058 entries, 0 to 155057
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   head_entity  155058 non-null  object
 1   relation     155058 non-null  object
 2   tail_entity  155058 non-null  object
dtypes: object(3)
memory usage: 3.5+ MB


In [6]:
# load each split
HW5M3MIL_test = pd.read_csv('/home/lena/git/master_thesis_bias_in_NLP/data/processed/output_of_preprocessing/test_data_subset_0.05_rs42_06_05_2022_15:11.tsv',
                         sep = '\t', names = ['head_entity', 'relation', 'tail_entity'])
HW5M3MIL_test.head()
HW5M3MIL_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155058 entries, 0 to 155057
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   head_entity  155058 non-null  object
 1   relation     155058 non-null  object
 2   tail_entity  155058 non-null  object
dtypes: object(3)
memory usage: 3.5+ MB


In [29]:
# how many female screenwriters in train?

# 1. extract the screenwriters
# relation: occupation = P106
# tail screenwriter = Q28389
screenwriters_train = HW5M3MIL_train[(HW5M3MIL_train['relation'] == 'P106') & (HW5M3MIL_train['tail_entity'] == 'Q28389')]
screenwriters_train.info()
# 5685 in total
set_of_screenwriters_train = set(screenwriters_train['head_entity'])
len(set_of_screenwriters_train)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5685 entries, 427 to 2789219
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   head_entity  5685 non-null   object
 1   relation     5685 non-null   object
 2   tail_entity  5685 non-null   object
dtypes: object(3)
memory usage: 177.7+ KB


5685

In [40]:
# 2. which of these are female?
female_gender_entity = 'Q6581072'
male_gender_entity = 'Q6581097'
gender_relation = 'P21'

screenwriter_all_facts_mask = HW5M3MIL_train['head_entity'].isin(set_of_screenwriters_train)
screenwriter_all_facts = HW5M3MIL_train[screenwriter_all_facts_mask]

screenwriter_female_gender_facts = screenwriter_all_facts[(screenwriter_all_facts['relation'] == gender_relation)]#screenwriter_all_facts['tail_entity'] == male_gender_entity]
screenwriter_female_gender_facts['tail_entity'].value_counts()

Q6581097    1087
Q6581072     199
Name: tail_entity, dtype: int64

In [38]:
screenwriter_female_gender_facts

,head_entity,relation,tail_entity
1329373,Q2357826,P21,Q6581097
1329468,Q1702239,P21,Q6581097
1329952,Q3161467,P21,Q6581097
1331079,Q16556648,P21,Q6581097
1331956,Q18062161,P21,Q6581097
...,...,...,...
2786488,Q3953024,P21,Q6581097
2786528,Q275920,P21,Q6581072
2788954,Q4662138,P21,Q6581097
2789897,Q361981,P21,Q6581097


In [41]:
# how many female screenwriters in val?

# 1. extract the screenwriters
# relation: occupation = P106
# tail screenwriter = Q28389
screenwriters_val = HW5M3MIL_val[(HW5M3MIL_val['relation'] == 'P106') & (HW5M3MIL_val['tail_entity'] == 'Q28389')]
screenwriters_val.info()
# 5685 in total
set_of_screenwriters_val = set(screenwriters_val['head_entity'])
len(set_of_screenwriters_val)
# 2. which of these are female?
female_gender_entity = 'Q6581072'
male_gender_entity = 'Q6581097'
gender_relation = 'P21'

screenwriter_all_facts_mask = HW5M3MIL_val['head_entity'].isin(set_of_screenwriters_val)
screenwriter_all_facts = HW5M3MIL_val[screenwriter_all_facts_mask]

screenwriter_female_gender_facts = screenwriter_all_facts[(screenwriter_all_facts['relation'] == gender_relation)]#screenwriter_all_facts['tail_entity'] == male_gender_entity]
screenwriter_female_gender_facts['tail_entity'].value_counts()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 105 entries, 1341 to 154417
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   head_entity  105 non-null    object
 1   relation     105 non-null    object
 2   tail_entity  105 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB


Q6581097    2
Name: tail_entity, dtype: int64

In [42]:
# how many female screenwriters in test?

# 1. extract the screenwriters
# relation: occupation = P106
# tail screenwriter = Q28389
screenwriters_test = HW5M3MIL_test[(HW5M3MIL_test['relation'] == 'P106') & (HW5M3MIL_test['tail_entity'] == 'Q28389')]
screenwriters_test.info()
# 5685 in total
set_of_screenwriters_test = set(screenwriters_test['head_entity'])
len(set_of_screenwriters_test)
# 2. which of these are female?
female_gender_entity = 'Q6581072'
male_gender_entity = 'Q6581097'
gender_relation = 'P21'

screenwriter_all_facts_mask = HW5M3MIL_test['head_entity'].isin(set_of_screenwriters_test)
screenwriter_all_facts = HW5M3MIL_test[screenwriter_all_facts_mask]

screenwriter_female_gender_facts = screenwriter_all_facts[(screenwriter_all_facts['relation'] == gender_relation)]#screenwriter_all_facts['tail_entity'] == male_gender_entity]
screenwriter_female_gender_facts['tail_entity'].value_counts()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 12 to 151866
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   head_entity  106 non-null    object
 1   relation     106 non-null    object
 2   tail_entity  106 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB


Q6581097    1
Name: tail_entity, dtype: int64

In [ ]:
# And how many are there in the

In [43]:
HW5M11mil = pd.read_csv('/home/lena/git/master_thesis_bias_in_NLP/data/interim/wikidata5m_human_facts_with_binary_gender_added_01042022_v3.tsv',
                         sep = '\t', names = ['head_entity', 'relation', 'tail_entity'])
HW5M11mil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11114797 entries, 0 to 11114796
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   head_entity  object
 1   relation     object
 2   tail_entity  object
dtypes: object(3)
memory usage: 254.4+ MB


In [44]:
# how many female screenwriters in HW5M11mil?

# 1. extract the screenwriters
# relation: occupation = P106
# tail screenwriter = Q28389
screenwriters_HW5M11mil = HW5M11mil[(HW5M11mil['relation'] == 'P106') & (HW5M11mil['tail_entity'] == 'Q28389')]
screenwriters_HW5M11mil.info()
# 5685 in total
set_of_screenwriters_HW5M11mil = set(screenwriters_HW5M11mil['head_entity'])
len(set_of_screenwriters_HW5M11mil)
# 2. which of these are female?
female_gender_entity = 'Q6581072'
male_gender_entity = 'Q6581097'
gender_relation = 'P21'

screenwriter_all_facts_mask = HW5M11mil['head_entity'].isin(set_of_screenwriters_HW5M11mil)
screenwriter_all_facts = HW5M11mil[screenwriter_all_facts_mask]

screenwriter_female_gender_facts = screenwriter_all_facts[(screenwriter_all_facts['relation'] == gender_relation)]#screenwriter_all_facts['tail_entity'] == male_gender_entity]
screenwriter_female_gender_facts['tail_entity'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21146 entries, 548 to 9612760
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   head_entity  21146 non-null  object
 1   relation     21146 non-null  object
 2   tail_entity  21146 non-null  object
dtypes: object(3)
memory usage: 660.8+ KB


Q6581097    17778
Q6581072     3362
Name: tail_entity, dtype: int64

In [45]:
# female screenwriter percentage in 11mil
3362/(3362+17778)


0.15903500473036897

In [46]:
# male screenwriter percentage in 11mil
17778/(3362+17778)

0.840964995269631

In [47]:
# female screenwriter percentage in 3mil
199/(1087+199)

0.1547433903576983

In [48]:
# female screenwriter percentage in 3mil
1087/(1087+199)

0.8452566096423018